<a href="https://colab.research.google.com/github/Menace41001/Fine_tuned_Model/blob/main/fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
!pip install --no-deps xformers trl peft accelerate bitsandbytes


In [ ]:
!pip install "unsloth[colab-new]"
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


In [ ]:
from datasets import load_dataset
expert_constitution_prompt = """You are an expert on the Constitution of India. Your task is to answer the following question directly and accurately based on constitutional principles.

### Question:
{}

### Answer:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    questions = examples["question"]
    answers   = examples["answer"]
    texts = []
    for question, answer in zip(questions, answers):
        text = expert_constitution_prompt.format(question, answer) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# Load and map the dataset
dataset = load_dataset("json", data_files="/content/drive/MyDrive/Colab Notebooks/finetuning_dataset/constitution_qa.json", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

In [ ]:
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
print(f"Baseline GPU memory usage: {start_gpu_memory} GB")

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 2,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:

max_memory = round(torch.cuda.get_device_properties(0).total_memory / 1024**3, 3)

# Calculate the final stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024**3, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

# Print the results
print(f"--- Training Stats ---")
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
FastLanguageModel.for_inference(model)

inputs = tokenizer(
[
    expert_constitution_prompt.format(
        "What does the Right to Equality state in the Constitution of India?",
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 210, use_cache = True)
print(tokenizer.batch_decode(outputs)[0])

In [ ]:
save_directory = "/content/drive/MyDrive/Colab Notebooks/Lora_finetuned_model/consitution_lora_adapter"
model.save_pretrained(save_directory)
print(f"Model and tokenizer saved successfully to: {save_directory}")

In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/content/drive/MyDrive/Colab Notebooks/Lora_finetuned_model/consitution_lora_adapter",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference


inputs = tokenizer(
[
    expert_constitution_prompt.format(

        "Who has the power to grant pardons, reprieves, or remissions of punishment?",
        "",
    )
], return_tensors = "pt").to("cuda")

# Generate the response
outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
print(tokenizer.batch_decode(outputs)[0])

In [ ]:
# change to true to save the model combined with lora adapters
final_dir = "/content/drive/MyDrive/Colab Notebooks/Merged_model/constitution_model"
if False: model.save_pretrained_merged(final_dir, tokenizer, save_method = "merged_16bit",)